# Basic Examples for SKlearn Prepackaged Server with MinIO


## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb) to setup Seldon Core with an ingress.

## Get MinIO CLI client tool

Using go get:
```bash   
GO111MODULE=on go get github.com/minio/mc
```

Or follow steps relevant to your platform from official [documentation](https://docs.min.io/docs/minio-client-quickstart-guide.html).

## Install MinIO in cluster

### Helm install minio

In [1]:
%%bash
kubectl create ns minio-system
helm install minio stable/minio \
    --set accessKey=minioadmin \
    --set secretKey=minioadmin \
    --namespace minio-system

namespace/minio-system created
NAME: minio
LAST DEPLOYED: Wed Apr 22 17:57:31 2020
NAMESPACE: minio-system
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Minio can be accessed via port 9000 on the following DNS name from within your cluster:
minio.minio-system.svc.cluster.local

To access Minio from localhost, run the below commands:

  1. export POD_NAME=$(kubectl get pods --namespace minio-system -l "release=minio" -o jsonpath="{.items[0].metadata.name}")

  2. kubectl port-forward $POD_NAME 9000 --namespace minio-system

Read more about port forwarding here: http://kubernetes.io/docs/user-guide/kubectl/kubectl_port-forward/

You can now access Minio server on http://localhost:9000. Follow the below steps to connect to Minio server with mc client:

  1. Download the Minio mc client - https://docs.minio.io/docs/minio-client-quickstart-guide

  2. mc config host add minio-local http://localhost:9000 minioadmin minioadmin S3v4

  3. mc ls minio-local

Alternately, you can use your

In [2]:
!kubectl rollout status deployment -n minio-system minio 

Waiting for deployment "minio" rollout to finish: 0 of 1 updated replicas are available...
deployment "minio" successfully rolled out


### port-forward Minio to localhost

in separate terminal:

```bash
kubectl port-forward -n minio-system svc/minio 9000:9000
```

or follow instructions printed by helm

### Configure mc client

In [4]:
%%bash
mc config host add minio-local http://localhost:9000 minioadmin minioadmin

Added `minio-local` successfully.


## Copy iris model into local MinIO

In [5]:
%%bash
mc config host add gcs https://storage.googleapis.com "" "" 

mc mb minio-local/iris -p
mc cp gcs/seldon-models/sklearn/iris/model.joblib minio-local/iris/
mc cp gcs/seldon-models/sklearn/iris/metadata.yaml minio-local/iris/

Added `gcs` successfully.
Bucket created successfully `minio-local/iris`.
`gcs/seldon-models/sklearn/iris/model.joblib` -> `minio-local/iris/model.joblib`
Total: 0 B, Transferred: 1.06 KiB, Speed: 660 B/s
`gcs/seldon-models/sklearn/iris/metadata.yaml` -> `minio-local/iris/metadata.yaml`
Total: 0 B, Transferred: 162 B, Speed: 96 B/s


## Modify model metadata (optional)

In [6]:
%%bash
mc cat minio-local/iris/metadata.yaml


name: iris
versions: [iris/v1]
platform: sklearn
inputs:
- datatype: BYTES
  name: input
  shape: [ 4 ]
outputs:
- datatype: BYTES
  name: output
  shape: [ 3 ]


In [7]:
%%writefile metadata.yaml

name: iris
versions: [iris/v1-updated]
platform: sklearn
inputs:
- datatype: BYTES
  name: input
  shape: [ 1, 4 ]
outputs:
- datatype: BYTES
  name: output
  shape: [ 3 ]

Overwriting metadata.yaml


In [8]:
%%bash
mc cp metadata.yaml minio-local/iris/

`metadata.yaml` -> `minio-local/iris/metadata.yaml`
Total: 0 B, Transferred: 173 B, Speed: 16.22 KiB/s


## Deploy sklearn server

In [9]:
%%writefile secret.yaml

apiVersion: v1
kind: Secret
metadata:
  name: seldon-init-container-secret
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: minioadmin
  AWS_SECRET_ACCESS_KEY: minioadmin
  AWS_ENDPOINT_URL: http://minio.minio-system.svc.cluster.local:9000
  USE_SSL: "false"

Overwriting secret.yaml


In [10]:
!kubectl apply -f secret.yaml

secret/seldon-init-container-secret created


In [11]:
%%writefile deploy.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: minio-sklearn
spec:
  annotations:
    seldon.io/executor: "true"
  name: iris
  predictors:
  - componentSpecs:
    graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: s3://iris
      envSecretRefName: seldon-init-container-secret
      name: classifier
    name: default
    replicas: 1

Overwriting deploy.yaml


In [12]:
!kubectl apply -f deploy.yaml

seldondeployment.machinelearning.seldon.io/minio-sklearn created


In [13]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=minio-sklearn -o jsonpath='{.items[0].metadata.name}')

deployment "minio-sklearn-default-0-classifier" successfully rolled out


## Test deployment

### Test prediction

In [14]:
%%bash
curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://localhost:8003/seldon/seldon/minio-sklearn/api/v1.0/predictions  | jq .

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "ndarray": [
      [
        0.9548873249364169,
        0.04505474761561406,
        5.7927447968952436e-05
      ]
    ]
  },
  "meta": {}
}


### Test model metadata (optional)

In [15]:
%%bash
curl -s http://localhost:8003/seldon/seldon/minio-sklearn/api/v1.0/metadata/classifier | jq .

{
  "inputs": [
    {
      "datatype": "BYTES",
      "name": "input",
      "shape": [
        1,
        4
      ]
    }
  ],
  "name": "iris",
  "outputs": [
    {
      "datatype": "BYTES",
      "name": "output",
      "shape": [
        3
      ]
    }
  ],
  "platform": "sklearn",
  "versions": [
    "iris/v1-updated"
  ]
}


## Cleanup

In [16]:
!kubectl delete -f deploy.yaml

seldondeployment.machinelearning.seldon.io "minio-sklearn" deleted
